# Create an LLM Agent Capable of Math and Machine Learning Operations, Using the OpenAI API.

You are an AI engineer working at a financial services firm. Your financial analysts want to use modern large language models as a tool, but need access to mathematical and regression tools that LLM’s are not natively skilled at. Your manager has asked you to prototype an LLM agent solution, coding directly against the OpenAI chat completions API, that augments a chatbot with local tools capable of performing basic mathematical operations and linear regression. Your LLM agent will be evaluated by its ability to successfully compute math and regression problems given as natural language prompts.

In [1]:
import json



In [2]:
def perform_math_operation(operation, args=[]):
    result = 0
    if operation == 'add':
        result = sum(args)
    elif operation == 'subtract':
        result = args[0] - sum(args[1:])
    elif operation == 'multiply':
        result = 1
        for num in args:
            result *= num
    elif operation == 'divide':
        result = args[0]
        for num in args[1:]:
            result /= num
    else:
        raise ValueError("Unsupported operation")
 
    retval = {"Result": result}
    return json.dumps(retval) 

In [3]:
perform_math_operation("multiply", [8,2,2])



'{"Result": 32}'

In [4]:
perform_math_operation("add", [8,2,2])



'{"Result": 12}'

In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [6]:
def predict_next_in_series(series):
    x = np.array(range(len(series))).reshape(-1, 1)
    y = np.array(series).reshape(-1, 1)

In [7]:
def predict_next_in_series(series):
    x = np.array(range(len(series))).reshape(-1, 1)
    y = np.array(series).reshape(-1, 1)
    model = LinearRegression().fit(x, y)

In [8]:
def predict_next_in_series(series):
    x = np.array(range(len(series))).reshape(-1, 1)
    y = np.array(series).reshape(-1, 1)
    model = LinearRegression().fit(x, y)
    next_index = len(series)
    next_value = model.predict([[next_index]])

In [9]:
def predict_next_in_series(series):
    x = np.array(range(len(series))).reshape(-1, 1)
    y = np.array(series).reshape(-1, 1)
    model = LinearRegression().fit(x, y)
    next_index = len(series)
    next_value = model.predict([[next_index]])
    next_info = {
        "Next value": next_value[0][0]
    }
    return json.dumps(next_info)

In [10]:
predict_next_in_series([2, 4, 6, 8])



'{"Next value": 10.0}'

In [11]:
mathTool = {
        "type": "function",
        "function": {
            "name": "perform_math_operation",
            "description": "Perform a mathematical operation",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": 
                    {
                        "type": "string", 
                        "enum": ["add", "subtract", "multiply", "divide"],
                        "description": "The requested mathematical operation to perform."
                    },
                    "operands": 
                    {
                        "type": "array", 
                        "items": {"type": "number"},
                        "description": "The operands of the requested mathematical operation, from left to right."
                    }
                },
                "required": ["operation", "operands"]
            },
        }
    }

In [12]:
regressionTool = {
        "type": "function",
        "function": {
            "name": "predict_next_in_series",
            "description": "Predict the next number in a series using linear regression",
            "parameters": {
                "type": "object",
                "properties": {
                    "series": {
                        "type": "array", 
                        "items": {"type": "number"},
                        "description": "The list of numbers in the series to be completed."
                    }
                },
                "required": ["series"]
            },
        }
    }

In [13]:
tools = [mathTool, regressionTool]

In [14]:
import openai
from openai import OpenAI
 
client = OpenAI()

In [15]:
def make_initial_request(messages):
 
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools,
        tool_choice=None
    )
    
    response_message = response.choices[0].message
    
    print(response_message)
    
    return response_message

In [16]:
def call_tool(tool):
    
    print("calling tool: ")
    print(tool)
    
    response = {}
    
    function_name = tool.function.name
    if (function_name == "perform_math_operation"):
        args = json.loads(tool.function.arguments)
        fn_response = perform_math_operation(operation=args.get("operation"),args=args.get("operands"))
        response = {
            "tool_call_id": tool.id,
            "role": "tool",
            "name": function_name,
            "content": fn_response,
        }
    elif (function_name == "predict_next_in_series"):
        args = json.loads(tool.function.arguments)
        fn_response = predict_next_in_series(series=args.get("series"))
        response = {
            "tool_call_id": tool.id,
            "role": "tool",
            "name": function_name,
            "content": fn_response,
        }
    return response

In [17]:
def llm_agent(prompt):
    messages = [{"role": "user", "content": prompt}]
    
    first_response = make_initial_request(messages)
    
    tool_calls = first_response.tool_calls
    if (tool_calls):
 
        messages.append(first_response)
        
        for tool in tool_calls:
            tool_response = call_tool(tool)
            messages.append(tool_response)
 
        print("second call:")
        print(messages)
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages = messages,
        )
            
        return second_response
    else:
        return response

In [18]:
prompt = "What is 5 divided by 3?"
print (llm_agent(prompt))

ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='', name=''), tool_calls=[ChatCompletionMessageToolCall(id='call_psU4MSP7ZEZljlwaIvLL0yNF', function=Function(arguments='{"operands":[5,3],"operation":"divide"}', name='perform_math_operation'), type='function')], name='')
calling tool: 
ChatCompletionMessageToolCall(id='call_psU4MSP7ZEZljlwaIvLL0yNF', function=Function(arguments='{"operands":[5,3],"operation":"divide"}', name='perform_math_operation'), type='function')
second call:
[{'role': 'user', 'content': 'What is 5 divided by 3?'}, ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='', name=''), tool_calls=[ChatCompletionMessageToolCall(id='call_psU4MSP7ZEZljlwaIvLL0yNF', function=Function(arguments='{"operands":[5,3],"operation":"divide"}', name='perform_math_operation'), type='function')], name=''), {'tool_call_id': 'call_psU4MSP7ZEZljlwaIvLL0yNF', 'r

In [19]:
prompt = "What is the next value in the series 1, 3, 5, 7, 11?"
print (llm_agent(prompt))

ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='', name=''), tool_calls=[ChatCompletionMessageToolCall(id='call_5RiCBtjGqw2IKI41sHPq5LKA', function=Function(arguments='{"series":[1,3,5,7,11]}', name='predict_next_in_series'), type='function')], name='')
calling tool: 
ChatCompletionMessageToolCall(id='call_5RiCBtjGqw2IKI41sHPq5LKA', function=Function(arguments='{"series":[1,3,5,7,11]}', name='predict_next_in_series'), type='function')
second call:
[{'role': 'user', 'content': 'What is the next value in the series 1, 3, 5, 7, 11?'}, ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=FunctionCall(arguments='', name=''), tool_calls=[ChatCompletionMessageToolCall(id='call_5RiCBtjGqw2IKI41sHPq5LKA', function=Function(arguments='{"series":[1,3,5,7,11]}', name='predict_next_in_series'), type='function')], name=''), {'tool_call_id': 'call_5RiCBtjGqw2IKI41sHPq5LKA', 'role': 'tool', 'name